In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googleapis/llama-index-cloud-sql-pg-python/blob/main/samples/llama_index_quick_start.ipynb)

---
# Introduction

In this codelab, you'll learn how to create a powerful interactive generative AI application using Retrieval Augmented Generation powered by [Cloud SQL for PostgreSQL](https://cloud.google.com/sql/docs/postgres) and [LlamaIndex](https://www.llamaindex.ai/). We will be creating an application grounded in a [Netflix Movie dataset](https://www.kaggle.com/datasets/shivamb/netflix-shows), allowing you to interact with movie data in exciting new ways.

## Prerequisites

* A basic understanding of the Google Cloud Console
* Basic skills in command line interface and Google Cloud shell
* Basic python knowledge

## What you'll learn

* How to deploy a Cloud SQL for PostgreSQL instance
* How to use Cloud SQL for PostgreSQL as a Document Reader
* How to use Cloud SQL for PostgreSQL as a Vector Store
* How to use Cloud SQL for PostgreSQL as a Document Store with a Index Store
* How to use Cloud SQL for PostgreSQL as a Chat Store

## What you'll need

* A Google Cloud Account and Google Cloud Project
* A web browser such as [Chrome](https://www.google.com/chrome/)

# Setup and Requirements

In the following instructions you will learn to:

1. Install required dependencies for our application
2. Set up authentication for our project
3. Set up a Cloud SQL for PostgreSQL Instance
4. Import the data used by our application

## Install dependencies
First you will need to install the dependencies needed to run this demo app.

In [ ]:
%pip install llama-index-cloud-sql-pg llama-index llama-index-embeddings-vertex llama-index-llms-vertex cloud-sql-python-connector[pg8000]

**Colab only:** Uncomment the following cell to restart the kernel or use the button to restart the kernel. For Vertex AI Workbench you can restart the terminal using the button on top.

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## Authenticate to Google Cloud within Colab
In order to access your Google Cloud Project from this notebook, you will need to Authenticate as an IAM user.

In [ ]:
from google.colab import auth

auth.authenticate_user()

## Connect Your Google Cloud Project

In [ ]:
# @markdown Please fill in the value below with your GCP project ID and then run the cell.

# Please fill in these values.
project_id = ""  # @param {type:"string"}

# Quick input validations.
assert project_id, "⚠️ Please provide a Google Cloud project ID"

# Configure gcloud.
!gcloud config set project {project_id}

## Configure Your Google Cloud Project

Configure the following in your Google Cloud Project.

You will need to enable these APIs in order to use `VertexTextEmbeddings` as an embeddings service!

1. IAM principal (user, service account, etc.) with the [Cloud SQL Client][client-role] role. The user logged into this notebook will be used as the IAM principal and will be granted the Cloud SQL Client role.

[client-role]: https://cloud.google.com/sql/docs/mysql/roles-and-permissions

In [ ]:
current_user = !gcloud auth list --filter=status:ACTIVE --format="value(account)"
!gcloud projects add-iam-policy-binding {project_id} \
  --member=user:{current_user[0]} \
  --role="roles/cloudsql.client"

2. Enable the APIs for Cloud SQL and Vertex AI within your project.

In [ ]:
# Enable GCP services
!gcloud services enable sqladmin.googleapis.com aiplatform.googleapis.com

## Set up Cloud SQL
You will need a **Postgres** Cloud SQL instance for the following stages of this notebook. Please set the following variables.

In [ ]:
# @markdown Please fill in the both the Google Cloud region and name of your Cloud SQL instance. Once filled in, run the cell.

# Please fill in these values.
region = "us-central1"  # @param {type:"string"}
instance_name = "llamaindex-quickstart-instance"  # @param {type:"string"}
database_name = "llamaindex-quickstart-db"  # @param {type:"string"}
user = "postgres"  # @param {type:"string"}
password = input("Please provide a password to be used for 'postgres' database user: ")

### Create a Postgres Instance
Running the below cell will verify the existence of the Cloud SQL instance and or create a new instance and database if one does not exist.

> ⏳ - Creating a Cloud SQL instance may take a few minutes.

In [ ]:
# check if Cloud SQL instance exists in the provided region
database_version = !gcloud sql instances describe {instance_name} --format="value(databaseVersion)"
if database_version[0].startswith("POSTGRES"):
  print("Found existing Postgres Cloud SQL Instance!")
else:
  print("Creating new Cloud SQL instance...")
  !gcloud sql instances create {instance_name} --database-version=POSTGRES_15 \
    --region={region} --cpu=1 --memory=4GB --root-password={password} \
    --database-flags=cloudsql.iam_authentication=On

### Create a Database

Next you will create database to store the data for this application.

In [ ]:
databases = !gcloud sql databases list --instance={instance_name} --format="value(name)"
if database_name not in databases:
    !gcloud sql databases create {database_name} --instance={instance_name}
else:
    print("Found existing database!")

#### Connect to our New Database

Now you will use `PostgresEngine` that connects to your new database!

In [ ]:
from google.cloud.sql.connector import Connector
import sqlalchemy

# initialize Connector object
connector = Connector()


# function to return the database connection
def getconn():
    conn = connector.connect(
        f"{project_id}:{region}:{instance_name}",
        "pg8000",
        user=user,
        password=password,
        db=database_name,
    )
    return conn


# create connection pool
pool = sqlalchemy.create_engine(
    "postgresql+pg8000://",
    creator=getconn,
)

## Import data to your database

Now that you have your database, you will need to import data! We will be using a [Netflix Dataset from Kaggle](https://www.kaggle.com/datasets/shivamb/netflix-shows). Here is what the data looks like:

| show_id | type    | title                | director         | cast                                                                                                                                                  | country       | date_added        | release_year | rating | duration  | listed_in                                    | description                                                                                                                                                                           |
|---------|---------|----------------------|------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------|---------------|-------------------|--------------|--------|-----------|----------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| s1      | Movie   | Dick Johnson Is Dead | Kirsten Johnson  |                                                                                                                                                        | United States | September 25, 2021 | 2020         | PG-13  | 90 min    | Documentaries                                | As her father nears the end of his life, filmmaker Kirsten Johnson stages his death in inventive and comical ways to help them both face the inevitable.                              |
| s2      | TV Show | Blood & Water        |                  | Ama Qamata, Khosi Ngema, Gail Mabalane, Thabang Molaba, Dillon Windvogel, Natasha Thahane, Arno Greeff, Xolile Tshabalala, Getmore Sithole, Cindy Mahlangu, Ryle De Morny, Greteli Fincham, Sello Maake Ka-Ncube, Odwa Gwanya, Mekaila Mathys, Sandi Schultz, Duane Williams, Shamilla Miller, Patrick Mofokeng | South Africa  | September 24, 2021 | 2021         | TV-MA  | 2 Seasons | International TV Shows, TV Dramas, TV Mysteries | After crossing paths at a party, a Cape Town teen sets out to prove whether a private-school swimming star is her sister who was abducted at birth.                                   |
| s3      | TV Show | Ganglands            | Julien Leclercq  | Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabiha Akkari, Sofia Lesaffre, Salim Kechiouche, Noureddine Farihi, Geert Van Rampelberg, Bakary Diombera                                   |               | September 24, 2021 | 2021         | TV-MA  | 1 Season  | Crime TV Shows, International TV Shows, TV Action & Adventure | To protect his family from a powerful drug lord, skilled thief Mehdi and his expert team of robbers are pulled into a violent and deadly turf war.                                     |
| s4      | TV Show | Jailbirds New Orleans |                  |                                                                                                                                                        |               | September 24, 2021 | 2021         | TV-MA  | 1 Season  | Docuseries, Reality TV                        | Feuds, flirtations and toilet talk go down among the incarcerated women at the Orleans Justice Center in New Orleans on this gritty reality series.                                   |
| s5      | TV Show | Kota Factory         |                  | Mayur More, Jitendra Kumar, Ranjan Raj, Alam Khan, Ahsaas Channa, Revathi Pillai, Urvi Singh, Arun Kumar                                                 | India        | September 24, 2021 | 2021         | TV-MA  | 2 Seasons | International TV Shows, Romantic TV Shows, TV Comedies | In a city of coaching centers known to train India’s finest collegiate minds, an earnest but unexceptional student and his friends navigate campus life. |


The following code has been prepared code to help insert the CSV data into your Cloud SQL for PostgreSQL database.

Download the CSV file:

In [ ]:
!gsutil cp gs://cloud-samples-data/llamaindex/netflix_titles.csv .

The download can be verified by the following command or using the "Files" tab.

In [ ]:
!ls

In this next step you will:

1. Create the table into store data
2. And insert the data from the CSV into the database table

> To avoid costs, the following code uses only 100 rows as an example

In [ ]:
import pandas as pd
import sqlalchemy

create_table_cmd = sqlalchemy.text(
    'CREATE TABLE netflix_titles ( \
    show_id VARCHAR, \
    type VARCHAR, \
    title VARCHAR, \
    director VARCHAR, \
    "cast" VARCHAR, \
    country VARCHAR, \
    date_added VARCHAR, \
    release_year INTEGER, \
    rating VARCHAR, \
    duration VARCHAR, \
    listed_in VARCHAR, \
    description TEXT \
    )',
)

netflix_data = "/content/netflix_titles.csv"

df = pd.read_csv(netflix_data)
insert_data_cmd = sqlalchemy.text(
    """
    INSERT INTO netflix_titles VALUES (:show_id, :type, :title, :director,
      :cast, :country, :date_added, :release_year, :rating,
      :duration, :listed_in, :description)
    """
)

parameter_map = [
    {
        "show_id": row["show_id"],
        "type": row["type"],
        "title": row["title"],
        "director": row["director"],
        "cast": row["cast"],
        "country": row["country"],
        "date_added": row["date_added"],
        "release_year": row["release_year"],
        "rating": row["rating"],
        "duration": row["duration"],
        "listed_in": row["listed_in"],
        "description": row["description"],
    }
    for index, row in df.head(100).iterrows()  # limit to 100 rows
]

with pool.connect() as db_conn:
    db_conn.execute(create_table_cmd)
    db_conn.execute(
        insert_data_cmd,
        parameter_map,
    )
    db_conn.commit()
connector.close()

#### Set LLM and embedding models globally for Llama Index components

In [ ]:
from llama_index.core import Settings
from llama_index.embeddings.vertex import VertexTextEmbedding
from llama_index.llms.vertex import Vertex
import google.auth

credentials, _ = google.auth.default()

Settings.embed_model = VertexTextEmbedding(
    model_name="text-embedding-005", project=project_id, credentials=credentials
)
Settings.llm = Vertex(model="gemini-2.0-flash-001", project=project_id)

## Use case 1: Cloud SQL for Postgres as a Document Reader

---



Now that you have data in your database, you are ready to use Cloud SQL for PostgreSQL as a [Document Reader](https://docs.llamaindex.ai/en/stable/module_guides/loading/connector/). This means you will pull data from the database and load it into memory as documents. These documents can be used to create a Vector Store.

First, create a connection to your Cloud SQL for PostgreSQL instance using the `PostgresEngine` class.

To connect to your Cloud SQL instance from this notebook, you will need to enable public IP on your instance. Alternatively, you can follow [these instructions](https://cloud.google.com/sql/docs/postgres/connect-to-instance-from-outside-vpc) to connect to an Cloud SQL for PostgreSQL instance with Private IP from outside your VPC. Learn more about [specifying IP types](https://github.com/GoogleCloudPlatform/cloud-sql-python-connector?tab=readme-ov-file#specifying-ip-address-type).


In [ ]:
from llama_index_cloud_sql_pg import PostgresEngine

engine = PostgresEngine.from_instance(
    project_id=project_id,
    instance=instance_name,
    region=region,
    database=database_name,
    user=user,
    password=password,
    ip_type="public",
)

The `PostgresReader` requires an `PostgresEngine` object to define the database connection and a `table_name` to define which data is to be retrieved. The `content_columns` argument can be used to define the columns that will be used as "content" in the document object we will later construct. The rest of the columns in that table will become the "metadata" associated with the documents.

In [ ]:
from llama_index_cloud_sql_pg import PostgresReader


table_name = "netflix_titles"
content_columns = ["title", "director", "cast", "description"]
metadata_columns = [
    "show_id",
    "type",
    "country",
    "date_added",
    "release_year",
    "rating",
    "duration",
    "listed_in",
]
reader = PostgresReader.create_sync(
    engine=engine,
    table_name=table_name,
    content_columns=content_columns,
    metadata_columns=metadata_columns,
)

Use method `load_data()` to pull documents from out database. You can see the documents from the database here.

In [ ]:
documents = reader.load_data()
print(f"Loaded {len(documents)} from the database. Examples:")
for doc in documents[:5]:
    print(doc)

Nice, you just used Cloud SQL for Postgres as a Document Reader!

## Use case 2: Cloud SQL for PostgreSQL as Vector Store

---


Now, you will learn how to put all of the documents into a [Vector Store](https://docs.llamaindex.ai/en/stable/module_guides/storing/vector_stores/) so that you perform a vector search.

### Create Your Vector Store table

Create a Vector Store table that can preserve the Document's metadata by using the method `init_vector_store_table` and defining specific metadata columns. The vector size is required. The example shows the vector size, `768`, that corresponds with the length of the vectors computed by the model our embeddings service uses, Vertex AI's `text-embedding-005`. 

In [ ]:
from llama_index_cloud_sql_pg import Column

sample_vector_table_name = "movie_vector_table_samples"
engine.init_vector_store_table(
    sample_vector_table_name,
    vector_size=768,
    metadata_columns=[
        Column("show_id", "VARCHAR", nullable=True),
        Column("type", "VARCHAR", nullable=True),
        Column("country", "VARCHAR", nullable=True),
        Column("date_added", "VARCHAR", nullable=True),
        Column("release_year", "INTEGER", nullable=True),
        Column("rating", "VARCHAR", nullable=True),
        Column("duration", "VARCHAR", nullable=True),
        Column("listed_in", "VARCHAR", nullable=True),
    ],
    overwrite_existing=True,  # Enabling this will delete and recreate the table if exists.
)

### Create the Vector Store instance

Next, you will create a `PostgresVectorStore` object that connects to the new Cloud SQL database table to store the data from the documents.

In [ ]:
from llama_index_cloud_sql_pg import PostgresVectorStore

vector_store = PostgresVectorStore.create_sync(
    engine=engine,
    table_name=sample_vector_table_name,
    metadata_columns=[
        "show_id",
        "type",
        "country",
        "date_added",
        "release_year",
        "rating",
        "duration",
        "listed_in",
    ],
)

#### Create a LlamaIndex `Index`

An `Index` allows us to quickly retrieve relevant context for a user query. They are used to build `QueryEngines` and `ChatEngines` over which a user can get answers to their queries.
For a list of indexes that can be built in LlamaIndex, see [Index Guide](https://docs.llamaindex.ai/en/stable/module_guides/indexing/index_guide/).

A `VectorStoreIndex`, can be built using the `PostgresVectorStore`. You can also use the `PostgresDocumentStore` and `PostgresIndexStore` to persist documents and index metadata. These modules can be used to build other `Indexes`.
For a detailed python notebook on this, see [LlamaIndex Document Store Guide](https://github.com/googleapis/llama-index-cloud-sql-pg-python/blob/main/samples/llama_index_doc_store.ipynb).

Now, add the documents data into the vector table. Here is a code example to load the first 100 documents in the list.

In [ ]:
from llama_index.core import StorageContext, VectorStoreIndex

storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, show_progress=True
)

# If you have more documents, you can still add them to the database Vector Store table using the embedding service to create embeddings for each record
# vector_store.add_documents(docs_to_load)

#### Query over indexed data

A query engine takes in a natural language query and returns a rich response. It is built on one of the indexes, see [Query Engine Guide](https://docs.llamaindex.ai/en/stable/module_guides/deploying/query_engine/).

You can compose multiple query engines to achieve more advanced querying, see [Query Engine usage patterns](https://docs.llamaindex.ai/en/stable/module_guides/deploying/query_engine/usage_pattern/).

In [ ]:
query_engine = index.as_query_engine()
query = query_engine.query("List shows that are about teenagers")
query.response

# Use case 3: Cloud SQL for PostgreSQL as a Document Store with a Index Store

---


Llama Index breaks down documents into smaller units called nodes, storing them in a [Document Store](https://docs.llamaindex.ai/en/stable/module_guides/storing/docstores/). The Document Store can be used with multiple indexes where each index is stored in its own [Index Store](https://docs.llamaindex.ai/en/stable/module_guides/storing/index_stores/) and uses the same underlying nodes but can provide a different search capability.

### Set up a Document Store

In [ ]:
from llama_index_cloud_sql_pg import PostgresDocumentStore

document_store_table_name = "document_store"
engine.init_doc_store_table(table_name=document_store_table_name)
doc_store = PostgresDocumentStore.create_sync(
    engine=engine,
    table_name=document_store_table_name,
)

#### Parse documents into nodes

Using a `TokenTextSplitter`, you can split the movie names on whitespace characters for building a keyword based search index.

In [ ]:
from llama_index.core.node_parser import TokenTextSplitter

splitter = TokenTextSplitter(
    chunk_size=1024,
    chunk_overlap=20,
    separator=" ",
)
nodes = splitter.get_nodes_from_documents(documents)

#### Add nodes to Document Store

In [ ]:
from llama_index.core import StorageContext

storage_context = StorageContext.from_defaults(docstore=doc_store)
storage_context.docstore.add_documents(nodes)

### Set up an Index Store

In [ ]:
from llama_index_cloud_sql_pg import PostgresIndexStore

index_store_table_name = "index_store"
engine.init_index_store_table(
    table_name=index_store_table_name,
)

index_store = PostgresIndexStore.create_sync(
    engine=engine,
    table_name=index_store_table_name,
)

#### Create a Storage Context

In [ ]:
from llama_index.core import StorageContext

storage_context = StorageContext.from_defaults(
    docstore=doc_store, index_store=index_store
)

### Create Indexes

The Document Store can be used with multiple indexes. Each index uses the same underlying nodes. For example, the keyword table index extracts keywords from each Node and builds a mapping to all nodes containing that keyword. Let's use this to build a keyword search.

In [ ]:
from llama_index.core import SimpleKeywordTableIndex

keyword_table_index = SimpleKeywordTableIndex(nodes, storage_context=storage_context)

### Query the index

In [ ]:
query_engine = keyword_table_index.as_query_engine()
response = query_engine.query("What tv shows resonate with crime?")
print(response)

# Use case 4: Cloud SQL for PostgreSQL as Chat Store

---


Next, create a [Chat Store](https://docs.llamaindex.ai/en/stable/module_guides/storing/chat_stores/) so the LLM can retain context and information across multiple interactions, leading to more coherent and sophisticated conversations or text generation.

In [ ]:
from llama_index_cloud_sql_pg import PostgresChatStore

chat_store_table_name = "chat_store"
engine.init_chat_store_table(table_name=chat_store_table_name)
chat_store = PostgresChatStore.create_sync(
    engine,
    table_name=chat_store_table_name,
)

### Create a ChatMemoryBuffer
The `ChatMemoryBuffer` stores a history of recent chat messages, enabling the LLM to access relevant context from prior interactions.

By providing our Chat Store into the `ChatMemoryBuffer`, it can automatically retrieve and update messages associated with a specific session ID or `chat_store_key`.

In [ ]:
from llama_index.core.memory import ChatMemoryBuffer

memory = ChatMemoryBuffer.from_defaults(
    token_limit=3000,
    chat_store=chat_store,
    chat_store_key="user1",
)

### Create a LlamaIndex `ChatEngine`

You can re-use the `VectorStoreIndex` created above to create a [ChatEngine](https://docs.llamaindex.ai/en/stable/module_guides/deploying/chat_engines/) which includes a `ChatStore` to save the chats between the user and AI assistant.

Here is an example of how you would add a chat message and fetch all messages from the Chat Store.

In [ ]:
chat_engine = index.as_chat_engine(chat_mode="context", memory=memory)

op = chat_engine.chat("What was the cast in Blood and Water?")
op = chat_engine.chat("How many seasons are there for Kota Factory?")
# Retrieve response for a chat message
print(op.response)

# Retrieve all messages for a user / session
retrieved_messages = chat_store.get_messages("user1")
for msg in retrieved_messages:
    print(f"{msg.role} -> {msg.content}")